# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Send HTTP request to the target Website

In [2]:
url = "https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/"
response = requests.get(url)

check the response status code

In [3]:
response.status_code

200

### Parse the response content with BeautifulSoap

In [6]:
raw_html = response.text
raw_html
soup = BeautifulSoup(raw_html, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html lang="mk-MK" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">
 <head>
  <script data-no-optimize="1">
   var litespeed_docref=sessionStorage.getItem("litespeed_docref");litespeed_docref&&(Object.defineProperty(document,"referrer",{get:function(){return litespeed_docref}}),sessionStorage.removeItem("litespeed_docref"));
  </script>
  <meta charset="utf-8"/>
  <style id="litespeed-ccss">
   .br_alabel>span{padding:0;font-weight:700;display:inline-block;overflow:visible;text-overflow:clip;text-align:center;box-sizing:border-box;border:0 solid transparent;position:relative;-webkit-box-orient:vertical;-webkit-box-direction:normal;-ms-flex-direction:column;flex-direction:column;-webkit-box-pack:center;-ms-flex-pack:center;justify-content:center}.br_alabel.br_alabel_template_type_image>span{background-color:transparent!important;background-repeat:no-repeat;background-position:right top;background-size:contain}.br_alabel.br_alabel_template_type_image>span b{displa

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

In [11]:
product_title = soup.select_one('.wd-entities-title').text.strip()
product_title

'Oukitel WP20 Pro'

In [109]:
product_regular_price = soup.select_one('.woocommerce-Price-amount').text
currency = soup.select_one('.woocommerce-Price-amount').text.strip().split()[1]
price = soup.select_one('.woocommerce-Price-amount').text.strip().split()[0]
print(price, currency)
print(product_regular_price.strip())

0 ден
0 ден


In [106]:
p1 = soup.select('.price')
discount_prices_tags = []
for price in p1:
    if price.find('ins'):
        discount_prices_tags.append(price)

discount_price = []
for dpt in discount_prices_tags:
    if dpt.find('ins'):
        ins = dpt.find('ins')
        discount_price.append(ins)
#discount_prices_tags      
discount_price
for dp in discount_price:
    price = dp.text
    print(price)



15.999 ден
48.490 ден


In [60]:
product_discount = soup.findAll('del')
product_discount

[<del aria-hidden="true"><span class="woocommerce-Price-amount amount"><bdi>18.999 <span class="woocommerce-Price-currencySymbol">ден</span></bdi></span></del>,
 <del aria-hidden="true"><span class="woocommerce-Price-amount amount"><bdi>54.490 <span class="woocommerce-Price-currencySymbol">ден</span></bdi></span></del>]

In [28]:
product_link = soup.select_one('.product-image-link').get('href')
product_link

'https://clevershop.mk/product/acer-a315-23-a7kd/'

In [32]:
cart_button = soup.select_one('.wd-header-cart')
cart_button_a_link = cart_button.find('a')
cart_button_a_link_href = cart_button_a_link.get('href')
cart_button_a_link_href

'https://clevershop.mk/cart/'

Repeat the extraction process for each page of products

In [201]:
discount_price = []
prices = []
titles = []
urls = []

In [202]:
i = 1
while True:
    url = f"https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/{i}/"
    # scrape_all_pages(url);
    
    response = requests.get(url)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.text, "html.parser")
    product_title = soup.select_one('.wd-entities-title').text.strip()
    
    product_regular_price = soup.select_one('.woocommerce-Price-amount')
    regular_price = product_regular_price.text.strip()
    #print(f"product_title: ${i} : ${product_regular_price}")
    
    prices_all = soup.select('.price')
    
    
    discount_price_tags = []
    for price in prices_all:
        if price.find('ins'):
            discount_price_tags.append(price)

    discount_price_tag = soup.select_one('ins .woocommerce-Price-amount')
    dp = discount_price_tag.text.strip() if discount_price_tag else None
    
   

    titles.append(product_title)
    prices.append(regular_price)
    discount_price.append(dp)
    urls.append(url)
    
    
    i += 1

In [195]:
len(titles), len(prices), len(discount_price), len(urls)

(14, 14, 14, 14)

### Create a pandas DataFrame with the scraped products

In [203]:
data = {
    "Titles": titles,
    "Prices": prices,
    "Discount Prices": discount_price,
    "URLs" : urls,
}

In [204]:
df = pd.DataFrame(data)
df

,Titles,Prices,Discount Prices,URLs
0,Oukitel WP20 Pro,0 ден,15.999 ден,https://clevershop.mk/product-category/mobilni...
1,Oukitel WP20 Pro,0 ден,31.990 ден,https://clevershop.mk/product-category/mobilni...
2,Oukitel WP20 Pro,0 ден,47.990 ден,https://clevershop.mk/product-category/mobilni...
3,Oukitel WP20 Pro,0 ден,None,https://clevershop.mk/product-category/mobilni...
4,Oukitel WP20 Pro,0 ден,None,https://clevershop.mk/product-category/mobilni...
5,Oukitel WP20 Pro,0 ден,9.990 ден,https://clevershop.mk/product-category/mobilni...
6,Oukitel WP20 Pro,0 ден,None,https://clevershop.mk/product-category/mobilni...
7,Oukitel WP20 Pro,0 ден,49.990 ден,https://clevershop.mk/product-category/mobilni...
8,Oukitel WP20 Pro,0 ден,8.490 ден,https://clevershop.mk/product-category/mobilni...
9,Oukitel WP20 Pro,0 ден,19.990 ден,https://clevershop.mk/product-category/mobilni...


Save the dataframe as `.csv`

In [205]:
df.to_csv("lab1-shop.csv", index=False)